In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import json

In [34]:
driver = webdriver.Chrome()

In [35]:
driver.get("https://room.chuo-u.ac.jp/ct/home")

In [36]:
driver.find_element(By.ID, "username_input").send_keys("23D7104001I")
driver.find_element(By.ID, "password_input").send_keys("1025t1025")
driver.find_element(By.ID, "login_button").click()
time.sleep(3)



In [37]:
driver.get("https://room.chuo-u.ac.jp/ct/home")
html = driver.page_source

In [41]:
def parse_timetable_from_manaba(html: str):
    from bs4 import BeautifulSoup

    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find("table", class_="stdlist")

    # ヘッダーから曜日のリストを取得
    days = []
    for th in table.find("tr", class_="title").find_all("th")[1:]:
        days.append(th.text.strip())

    timetable_data = []

    # 時限ごとの行を解析
    for tr in table.find_all("tr")[1:]:
        tds = tr.find_all("td")
        if not tds:
            continue

        period = tds[0].text.strip()

        for i, td in enumerate(tds[1:]):
            course_div = td.find("div", class_="courselistweekly-c")
            if course_div:
                a_tag = course_div.find("a", href=True)
                if a_tag:
                    title = a_tag.text.strip()
                    href = a_tag['href']
                    has_unsubmitted = any(
                        "未提出の課題があります" in img.get("alt", "") 
                        for img in course_div.find_all("img")
                    )
                    timetable_data.append({
                        "day": days[i],
                        "period": int(period) if period.isdigit() else period,
                        "title": title,
                        "href": href,
                        "has_unsubmitted": has_unsubmitted
                    })

    return timetable_data

In [43]:
data = parse_timetable_from_manaba(html)

[{'day': '月',
  'period': 1,
  'title': '学問最前線',
  'href': 'course_5644371',
  'has_unsubmitted': True},
 {'day': '水',
  'period': 1,
  'title': '人工知能基礎論',
  'href': 'course_5646094',
  'has_unsubmitted': True},
 {'day': '月',
  'period': 2,
  'title': 'オペレーションズ・リサーチ１',
  'href': 'course_5646028',
  'has_unsubmitted': False},
 {'day': '火',
  'period': 2,
  'title': '応用解析１',
  'href': 'course_5645995',
  'has_unsubmitted': False},
 {'day': '水',
  'period': 2,
  'title': '信頼性工学',
  'href': 'course_5646055',
  'has_unsubmitted': False},
 {'day': '木',
  'period': 2,
  'title': '機械学習基礎論',
  'href': 'course_5646064',
  'has_unsubmitted': False},
 {'day': '金',
  'period': 2,
  'title': 'マーケティング・リサーチ',
  'href': 'course_5646043',
  'has_unsubmitted': False},
 {'day': '月',
  'period': 3,
  'title': 'ヒューマンメディア工学',
  'href': 'course_5646097',
  'has_unsubmitted': True},
 {'day': '火',
  'period': 3,
  'title': 'データサイエンス実験Ｂ',
  'href': 'course_5646010',
  'has_unsubmitted': False},
 {'day': '木',
  '